In [ ]:
from zod import ZodFrames
import zod.constants as constants

import pandas as pd

dataset_root = "/home/yshand/repos/cont-learning/data/zod"
version = "full" 


In [ ]:
zod_frames = ZodFrames(dataset_root=dataset_root, version=version)
training_frames = zod_frames.get_split(constants.TRAIN)
validation_frames = zod_frames.get_split(constants.VAL)

print(f"Number of training frames: {len(training_frames)}")
print(f"Number of validation frames: {len(validation_frames)}")

print("The 5 first training frames have the ids:", sorted(list(training_frames))[:5])

In [ ]:
def convert_zod_to_pandas(zod_dataset):
    
    data_list = []
    for frame in zod_frames:
        data_list.append(
            {
                "frame_id": frame.metadata.frame_id,
                "country_code": frame.metadata.country_code,
                "road_condition": frame.metadata.road_condition,
                "road_type": frame.metadata.road_type,
                "scraped_weather": frame.metadata.scraped_weather,
                "time_of_day": frame.metadata.time_of_day,
            }
        )
    return pd.DataFrame(data_list)

In [ ]:
df_train = convert_zod_to_pandas(training_frames)
df_val = convert_zod_to_pandas(validation_frames)